<a href="https://colab.research.google.com/github/bukhariaujih/KAU-training/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install pandas
# !pip install scikit-learn

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import random

import math

import itertools

import numpy as np

import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

import os

In [0]:
# setting the random seed for reproducibility
seed=42
random.seed(seed)
# dataset and model parameters
samples = 3625  # how many trials of eeg data
n_features = 32  # how many channels of eeg in each sample
time_steps = 1300 # how many ms was each sample run for

In [0]:
def one_hot_events(events):
    # helper function for one-hot encoding the events
    events_list = list(events)
    lb = preprocessing.LabelBinarizer()
    lb.fit(events_list)
    events_1hot = lb.transform(events_list)
    return events_1hot, lb


In [0]:
X = np.load('./drive/My Drive/training_1/Data/signal/X.npy')
y = np.load('./drive/My Drive/training_1/Data/signal/y_int.npy')

In [0]:
# removes the minor event types. There were only a couple hundred examples of each, whereas the used events had a 
# couple thousand examples
remove_list = [0,2,4,5,6]              # designate unwanted event types
# drop_list = np.isin(y, remove_list)    # create a list of indices associated with unwanted events                  
# drop_array = np.array(drop_list)       # make the list of indices to drop into an array

In [0]:
# make X, y's with the unwanted events removed
y_short_int = y[np.isin(y,remove_list, invert=True)]
X_short = X[np.isin(y, remove_list, invert=True)]

In [0]:
# one hot encode the y data without the unwanted events
y_short, _ = one_hot_events(y_short_int) 

In [0]:
# use strat. shuffle split to get indices for test and training data 
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=seed)
sss.get_n_splits(X_short, y_short)

2

In [0]:
# take the indices generated by stratified shuffle split and make the test and training datasets
for train_index, test_index in sss.split(X_short, y_short):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_short[train_index], X_short[test_index]
    y_train, y_test = y_short[train_index], y_short[test_index]

In [0]:
# Build LSTM with 100 neurons and dropout.
model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(time_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

In [0]:
!mkdir ./output

In [0]:
model.fit(X_train, y_train, batch_size=64, epochs=5)
  
model.save('./output/my_model.h5')  # Save model
del model  # Clear memory

In [0]:
model = load_model('./output/my_model.h5')
score = model.evaluate(X_test, y_test, batch_size=16)
score

In [0]:
print("Accuracy: %.2f%%" % (score[1]*100))